In [ ]:
"""
    list all files in dataframe with path and anomaly status
"""

import numpy as np
import pandas as pd

#import os
from os import listdir
from os.path import isfile, join
#import matplotlib.pyplot as plt

from joblib import dump, load
#import pickle

#from SoundFile import SoundFile

In [ ]:
from google.colab import drive
drive.mount('/drive') # 4/1AA...

Mounted at /drive


In [ ]:
def convertImage2DataframeDir(imageFolder):

    pngfiles = [f for f in listdir(imageFolder) if isfile(join(imageFolder, f))]

    i=0
    bilan = []

    for f in pngfiles:
        if f[-4:] != '.png':
            # ignore non .png files
            continue

        type = f.split('_')[0]

        if type == 'normal':
            anomaly = 0
        else:
            anomaly = 1

        imagePath = imageFolder + f

        #img = plt.imread(imagePath).T

        bilan.append((imagePath, anomaly))

    return pd.DataFrame(bilan, columns=['spectrogram', 'anomaly'])


In [ ]:
machines = ['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']
machines = ['pump']
#machines = ['ToyCar', 'ToyConveyor', 'valve']
sets = ['train', 'test']
sets = ['train']

dataFolder = '/drive/MyDrive/asdpy/data/original/dc2020task2'

for machine in machines:

    for s in sets:
        # example : '../data/fan/train_png/'
        imageFolder = dataFolder+'/'+machine+'/'+s+'_png/'

        print("Convert directory ", imageFolder)

        df = convertImage2DataframeDir(imageFolder)
        print(df.shape)
        print(df.head())

        saveName = dataFolder+'/'+machine+'/df_'+machine+'_'+s+'.joblib'
        dump(df, saveName, compress=True)

    # break


Convert directory  /drive/MyDrive/asdpy/data/original/dc2020task2/pump/train_png/
(3349, 2)
                                         spectrogram  anomaly
0  /drive/MyDrive/asdpy/data/original/dc2020task2...        0
1  /drive/MyDrive/asdpy/data/original/dc2020task2...        0
2  /drive/MyDrive/asdpy/data/original/dc2020task2...        0
3  /drive/MyDrive/asdpy/data/original/dc2020task2...        0
4  /drive/MyDrive/asdpy/data/original/dc2020task2...        0


In [ ]:
import matplotlib.pyplot as plt

def convertDataframe2Matrix(dataframe):

  # convertion des listes d'images en une matrice

  bilan = []
  anomalies = []
  i=0

  from time import time

  for ind in dataframe.index:
    imagePath = dataframe['spectrogram'][ind]
    anomaly = dataframe['anomaly'][ind]
    
    #print(imagePath)

    if (i % 250) == 1:
      print("{}/{}".format(i, len(dataframe)))

    t0 = time()
    img = plt.imread(imagePath).T
    t1 = time() - t0
    #print("imread en {} secondes".format(round(t1,3)))

    #print(img.T)
    #print(img.shape)
          
    # retaille l'image en un vecteur ligne de 40064 éléments
    img = img.reshape(img.shape[0]*img.shape[1])
    
    # ajoute le vecteur à la matrice de données

    t0 = time()
    bilan.append(img)
    anomalies.append(anomaly)
    t1 = time() - t0
    #print("append en {} secondes".format(round(t1,3)))
    i=i+1
    #print("====")


  return np.array(bilan), np.array(anomalies)



In [ ]:
#machines = ['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']
machines = ['valve']
#sets = ['train', 'test']
sets = ['train']

dataFolder = '/drive/MyDrive/asdpy/data/original/dc2020task2'

# va convertir les images en matrices
for machine in machines:

    for s in sets:
        saveName = dataFolder+'/'+machine+'/df_'+machine+'_'+s+'.joblib'
        dataframe = load(saveName)

        bilan, anomalies = convertDataframe2Matrix(dataframe)
        
        matrixSaveName = dataFolder+'/'+machine+'/df_matrices_'+machine+'_'+s+'.joblib'
        print(matrixSaveName)
        dump(bilan, matrixSaveName, compress=True)

        anomaliesSaveName = dataFolder+'/'+machine+'/df_matrices_anomalies_'+machine+'_'+s+'.joblib'
        print(anomaliesSaveName)
        dump(anomalies, anomaliesSaveName, compress=True)


1/3291
251/3291
501/3291
751/3291
1001/3291
1251/3291
1501/3291
1751/3291
2001/3291
2251/3291
2501/3291
2751/3291
3001/3291
3251/3291
/drive/MyDrive/asdpy/data/original/dc2020task2/valve/df_matrices_valve_train.joblib
/drive/MyDrive/asdpy/data/original/dc2020task2/valve/df_matrices_anomalies_valve_train.joblib


In [ ]:
import matplotlib.pyplot as plt

# conversions des différents fichiers mel en un unique dataframe
def convertMel2Dataframe(dataframe):

    files = [f for f in listdir(imageFolder) if isfile(join(imageFolder, f))]

    i=0
    bilan = []

    for f in files:
        print(f)
        if f[-12:] != '_mels.joblib':
            # ignore others files
            continue

        type = f.split('_')[0]

        if type == 'normal':
            anomaly = 0
        else:
            anomaly = 1

        #print("anomaly :", anomaly)

        filePath = imageFolder + f

        #img = plt.imread(filePath).T

        mels = load(filePath)

        # vectorize matrix
        vmels = np.reshape(mels, mels.shape[0]*mels.shape[1])

        line = [filePath] + vmels.tolist() + [anomaly]
        bilan.append(line)

    print(bilan)
      
    return pd.DataFrame(bilan)
    # return pd.DataFrame(bilan, columns=['spectrogram', 'anomaly'])



machines = ['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']
machines = ['fan']
#machines = ['ToyCar', 'ToyConveyor', 'valve']
sets = ['train', 'test']
#sets = ['train']

dataFolder = '/drive/MyDrive/asdpy/data/original/dc2020task2'

for machine in machines:

    for s in sets:
        # example : '../data/fan/train_png/'
        imageFolder = dataFolder+'/'+machine+'/'+s+'_png_128_64/'

        print("Convert directory ", imageFolder)

        df = convertMel2Dataframe(imageFolder)
        print(df.shape)
        print(df.head())

        saveName = dataFolder+'/'+machine+'/df_'+machine+'_128_64_'+s+'.joblib'
        dump(df, saveName, compress=True)

    # break



Streaming output truncated to the last 5000 lines.
normal_id_02_00000147_mels.joblib
normal_id_02_00000147.png
normal_id_02_00000129_mels.joblib
normal_id_02_00000129.png
normal_id_06_00000410.png
normal_id_02_00000137_mels.joblib
normal_id_02_00000137.png
normal_id_02_00000133_mels.joblib
normal_id_02_00000133.png
normal_id_06_00000423.png
normal_id_02_00000140_mels.joblib
normal_id_02_00000140.png
normal_id_02_00000152_mels.joblib
normal_id_02_00000152.png
normal_id_02_00000123_mels.joblib
normal_id_02_00000123.png
normal_id_02_00000111_mels.joblib
normal_id_02_00000111.png
normal_id_02_00000151_mels.joblib
normal_id_02_00000151.png
normal_id_02_00000109_mels.joblib
normal_id_02_00000109.png
normal_id_06_00000416.png
normal_id_02_00000139_mels.joblib
normal_id_02_00000139.png
normal_id_02_00000148_mels.joblib
normal_id_02_00000148.png
normal_id_06_00000414.png
normal_id_02_00000118_mels.joblib
normal_id_02_00000118.png
normal_id_02_00000112_mels.joblib
normal_id_02_00000112.png
norma

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(3675, 8194)
                                                0         1     ...      8192  8193
0  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.081388  ...  0.008966     0
1  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.091497  ...  0.004943     0
2  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.341905  ...  0.002035     0
3  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.112333  ...  0.002218     0
4  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.166616  ...  0.002301     0

[5 rows x 8194 columns]
Convert directory  /drive/MyDrive/asdpy/data/original/dc2020task2/fan/test_png_128_64/
anomaly_id_00_00000384_mels.joblib
anomaly_id_00_00000384.png
anomaly_id_02_00000007_mels.joblib
anomaly_id_02_00000007.png
anomaly_id_00_00000400_mels.joblib
anomaly_id_00_00000400.png
anomaly_id_00_00000401_mels.joblib
anomaly_id_00_00000401.png
anomaly_id_00_00000393_mels.joblib
anomaly_id_00_00000393.png
anomaly_id_00_00000372_mels.joblib
anomaly_id_00_00000372.png

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(1875, 8194)
                                                0         1     ...      8192  8193
0  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.156506  ...  0.001356     1
1  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.087758  ...  0.001331     1
2  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.109541  ...  0.002253     1
3  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.218811  ...  0.001151     1
4  /drive/MyDrive/asdpy/data/original/dc2020task2...  0.193618  ...  0.001182     1

[5 rows x 8194 columns]


Convert directory  /drive/MyDrive/asdpy/data/original/dc2020task2/fan/train_png_128_64/


KeyboardInterrupt: ignored